### 1. _Put phone in dev mode and installed Android ADB._
### 2. _Locate spotify usage and extract with ADB._
### 3. _Only one line of bash script really._
### 4. _Convert the txt to csv. et voila!._

In [1]:
import pprint
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt


# not scientific numbers
pd.set_option('display.float_format', '{:.5f}'.format)
np.set_printoptions(suppress=True)


### _why am i rewriting this over and over - use a function silly!!_

In [ ]:
def clean_usage_tables(file_path):
    # read - dropna - timestam to datetime - split to date & time columns sort byt date & time
    df = pd.read_csv(file_path)
    df = df.dropna()
    df['Timestamp'] = pd.to_datetime(df['Timestamp'])
    df['Date'] = df['Timestamp'].dt.date
    df['Time'] = df['Timestamp'].dt.time
    df = df.sort_values(by=['Date', 'Time'])

    return df

## _android spotify usage 1_

In [2]:
raw_df = pd.read_csv('data\my_spotify_usage_1.csv')

raw_df.columns

<>:1: SyntaxWarning: invalid escape sequence '\m'
<>:1: SyntaxWarning: invalid escape sequence '\m'
C:\Users\AARON\AppData\Local\Temp\ipykernel_1680\609058985.py:1: SyntaxWarning: invalid escape sequence '\m'
  raw_df = pd.read_csv('data\my_spotify_usage_1.csv')


Index(['Timestamp', 'Activity Type', 'Usage Duration (ms)'], dtype='object')

In [3]:
raw_df.dropna()

,Timestamp,Activity Type,Usage Duration (ms)
0,2025-02-16 11:31:05,USER_INTERACTION,com.spotify.music
1,2025-02-16 11:31:06,ACTIVITY_RESUMED,com.spotify.music
2,2025-02-16 11:31:24,FOREGROUND_SERVICE_START,com.spotify.music
3,2025-02-16 11:31:24,NOTIFICATION_INTERRUPTION,com.spotify.music
4,2025-02-16 11:31:27,ACTIVITY_PAUSED,com.spotify.music
...,...,...,...
118,2025-02-17 11:28:45,ACTIVITY_PAUSED,com.spotify.music
119,2025-02-17 11:28:45,ACTIVITY_RESUMED,com.spotify.music
120,2025-02-17 11:28:46,ACTIVITY_STOPPED,com.spotify.music
121,2025-02-17 11:28:46,ACTIVITY_PAUSED,com.spotify.music


* find first and last interaction per day

In [4]:
# convert to datetime
raw_df['Timestamp'] = pd.to_datetime(raw_df['Timestamp'])

#add date column
raw_df['Date'] = raw_df['Timestamp'].dt.date
raw_df['Time'] = raw_df['Timestamp'].dt.time


In [5]:
raw_df.head()

,Timestamp,Activity Type,Usage Duration (ms),Date,Time
0,2025-02-16 11:31:05,USER_INTERACTION,com.spotify.music,2025-02-16,11:31:05
1,2025-02-16 11:31:06,ACTIVITY_RESUMED,com.spotify.music,2025-02-16,11:31:06
2,2025-02-16 11:31:24,FOREGROUND_SERVICE_START,com.spotify.music,2025-02-16,11:31:24
3,2025-02-16 11:31:24,NOTIFICATION_INTERRUPTION,com.spotify.music,2025-02-16,11:31:24
4,2025-02-16 11:31:27,ACTIVITY_PAUSED,com.spotify.music,2025-02-16,11:31:27


In [6]:
#raw_df.drop('date', axis=1)

# sort timestamps
raw_df = raw_df.sort_values(by=['Date', 'Time'])


In [7]:
# group by date - find first and last

first_and_last = raw_df.groupby('Date').agg(
    first_interaction=('Time', 'first'),
    last_interaction=('Time', 'last')
).reset_index()

pprint.pprint(first_and_last)

         Date first_interaction last_interaction
0  2025-02-16          11:31:05         20:02:12
1  2025-02-17          00:09:46         11:28:46


* value counts for ACTIVITY_RESUMED, FOREGROUND_SERVICE_START, SHORTCUT_INVOCATION
    * display in chart

In [8]:
# filter for activity types
activity_filter = raw_df[raw_df['Activity Type'].isin(['ACTIVITY_RESUMED', 'FOREGROUND_SERVICE_START', 'SHORTCUT_INVOCATION'])]

In [9]:
# counts for each type
activity_count = activity_filter['Activity Type'].value_counts()

activity_count

Activity Type
ACTIVITY_RESUMED            28
FOREGROUND_SERVICE_START    10
SHORTCUT_INVOCATION          9
Name: count, dtype: int64

## _android - spotify usage 2_

In [10]:
raw_df_2 = pd.read_csv('data/my_spotify_usage_2.csv')

raw_df_2.columns

Index(['Timestamp', 'Activity Type', 'Usage Duration (ms)'], dtype='object')

In [11]:
raw_df_2.dropna()

,Timestamp,Activity Type,Usage Duration (ms)
0,2025-02-18 10:40:31,ACTIVITY_RESUMED,com.spotify.music
1,2025-02-18 10:40:40,ACTIVITY_PAUSED,com.spotify.music
2,2025-02-18 10:40:41,ACTIVITY_STOPPED,com.spotify.music
3,2025-02-18 11:27:08,FOREGROUND_SERVICE_STOP,com.spotify.music
4,2025-02-18 12:23:40,USER_INTERACTION,com.spotify.music
...,...,...,...
123,2025-02-19 10:15:32,FOREGROUND_SERVICE_START,com.spotify.music
124,2025-02-19 10:15:32,FOREGROUND_SERVICE_STOP,com.spotify.music
125,2025-02-19 10:25:48,FOREGROUND_SERVICE_STOP,com.spotify.music
126,2025-02-19 10:28:01,FOREGROUND_SERVICE_START,com.spotify.music


* find first and last interaction per day

In [12]:
# convert to datetime
raw_df_2['Timestamp'] = pd.to_datetime(raw_df['Timestamp'])

# add seperat time and date column
raw_df_2['Date'] = raw_df['Timestamp'].dt.date
raw_df_2['Time'] = raw_df['Timestamp'].dt.time

# sort timestamps
raw_df_2 = raw_df.sort_values(by=['Date', 'Time'])


In [13]:
#raw_df.drop('date', axis=1)

raw_df_2.head()


,Timestamp,Activity Type,Usage Duration (ms),Date,Time
0,2025-02-16 11:31:05,USER_INTERACTION,com.spotify.music,2025-02-16,11:31:05
1,2025-02-16 11:31:06,ACTIVITY_RESUMED,com.spotify.music,2025-02-16,11:31:06
2,2025-02-16 11:31:24,FOREGROUND_SERVICE_START,com.spotify.music,2025-02-16,11:31:24
3,2025-02-16 11:31:24,NOTIFICATION_INTERRUPTION,com.spotify.music,2025-02-16,11:31:24
4,2025-02-16 11:31:27,ACTIVITY_PAUSED,com.spotify.music,2025-02-16,11:31:27


In [14]:
# group by date - find first and last

first_and_last = raw_df_2.groupby('Date').agg(
    first_interaction=('Time', 'first'),
    last_interaction=('Time', 'last')
).reset_index()

pprint.pprint(first_and_last)

         Date first_interaction last_interaction
0  2025-02-16          11:31:05         20:02:12
1  2025-02-17          00:09:46         11:28:46
